In [ ]:
#write your names and students ids here

In [35]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
#from gensim.models import Word2Vec, KeyedVectors


from scipy.sparse import dok_matrix
# Install spaCy (run in terminal/prompt)
import sys
#!{sys.executable} -m pip install spacy
# Download spaCy's  'en' Model
#!{sys.executable} -m spacy download en



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bmsbu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
from sklearn import preprocessing

In [9]:
data=pd.read_json ('News_Category_Dataset_v2.json', lines=True)

### Typical data analysis of a dataframe now

In [10]:
data.shape

(200853, 6)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200853 entries, 0 to 200852
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   category           200853 non-null  object        
 1   headline           200853 non-null  object        
 2   authors            200853 non-null  object        
 3   link               200853 non-null  object        
 4   short_description  200853 non-null  object        
 5   date               200853 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.2+ MB


In [12]:
#data.describe
data.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [13]:
print(data.groupby('category').size())

category
ARTS               1509
ARTS & CULTURE     1339
BLACK VOICES       4528
BUSINESS           5937
COLLEGE            1144
COMEDY             5175
CRIME              3405
CULTURE & ARTS     1030
DIVORCE            3426
EDUCATION          1004
ENTERTAINMENT     16058
ENVIRONMENT        1323
FIFTY              1401
FOOD & DRINK       6226
GOOD NEWS          1398
GREEN              2622
HEALTHY LIVING     6694
HOME & LIVING      4195
IMPACT             3459
LATINO VOICES      1129
MEDIA              2815
MONEY              1707
PARENTING          8677
PARENTS            3955
POLITICS          32739
QUEER VOICES       6314
RELIGION           2556
SCIENCE            2178
SPORTS             4884
STYLE              2254
STYLE & BEAUTY     9649
TASTE              2096
TECH               2082
THE WORLDPOST      3664
TRAVEL             9887
WEDDINGS           3651
WEIRD NEWS         2670
WELLNESS          17827
WOMEN              3490
WORLD NEWS         2177
WORLDPOST          2579
dtype: 

In [9]:
#we can see that many categories are similar.
#Let´s merge them. We will merge the more specific ones into the more general ones in case of doubt
#the code in the cell below was obtained from https://www.kaggle.com/rmisra/news-category-dataset/discussion/114275

In [14]:
data['category']=data['category'].replace({"HEALTHY LIVING": "WELLNESS",
"QUEER VOICES": "GROUPS VOICES",
"BUSINESS": "BUSINESS & FINANCES",
"PARENTS": "PARENTING",
"BLACK VOICES": "GROUPS VOICES",
"THE WORLDPOST": "WORLD NEWS",
"STYLE": "STYLE & BEAUTY",
"GREEN": "ENVIRONMENT",
"TASTE": "FOOD & DRINK",
"WORLDPOST": "WORLD NEWS",
"SCIENCE": "SCIENCE & TECH",
"TECH": "SCIENCE & TECH",
"MONEY": "BUSINESS & FINANCES",
"ARTS": "ARTS & CULTURE",
"COLLEGE": "EDUCATION",
"LATINO VOICES": "GROUPS VOICES",
"CULTURE & ARTS": "ARTS & CULTURE",
"FIFTY": "MISCELLANEOUS",
"GOOD NEWS": "MISCELLANEOUS"})

In [15]:
print(data.groupby('category').size())

category
ARTS & CULTURE          3878
BUSINESS & FINANCES     7644
COMEDY                  5175
CRIME                   3405
DIVORCE                 3426
EDUCATION               2148
ENTERTAINMENT          16058
ENVIRONMENT             3945
FOOD & DRINK            8322
GROUPS VOICES          11971
HOME & LIVING           4195
IMPACT                  3459
MEDIA                   2815
MISCELLANEOUS           2799
PARENTING              12632
POLITICS               32739
RELIGION                2556
SCIENCE & TECH          4260
SPORTS                  4884
STYLE & BEAUTY         11903
TRAVEL                  9887
WEDDINGS                3651
WEIRD NEWS              2670
WELLNESS               24521
WOMEN                   3490
WORLD NEWS              8420
dtype: int64


In [16]:
(data.groupby('category').size()).std()

7299.533828002569

As we may observe, the data is still highly imbalanced: some categories contain about 1000 samples while others contain more than 10000. Therefore further techniques will have to be applied to
make up for this
(Using K-fold Cross-Validation?)

In [52]:
# import SMOTE module from imblearn library
# pip install imblearn (if you don't have imblearn in your system)
!conda install -c conda-forge imbalanced-learn
from imblearn.over_sampling import SMOTE
print('Original categories sizes %s' % data.groupby('category').size())
sm = SMOTE(random_state = 2)
data_res, y_res = sm.fit_sample(data, data['category'])
print('Resampled categories sizes %s' % data_res.groupby('category').size())

^C


ModuleNotFoundError: No module named 'imblearn'

## Data cleaning:

We must  apply either lemmatization or stemming to the words.
We will try to go for lemmatization first as it depends on the context of the word.

We used the following link to guide ourselves https://www.machinelearningplus.com/nlp/lemmatization-examples-python/


//TODO: preproces words to remove apostrophes for example. isn´t -> is not

In [17]:
lemmatizer = WordNetLemmatizer()

def clean(text):

    # Define the sentence to be lemmatized
    text=text.lower() #making everything lowercase
    
    # Tokenize: Split the sentence into words
    word_list = nltk.word_tokenize(text)
    #print(word_list)

    # Lemmatize list of words and join
    #lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    #print(lemmatized_output)
    
   # return lemmatized_output
    return word_list

In [18]:
data['short_description'] = data['short_description'].apply(clean)
data['headline'] = data['headline'].apply(clean)

### ENCODING CLASSES

We need to transform strings into numerical labels. Remember, if we extend our model to work with NN, 
then we should transform to one-hot encoding instead

In [19]:
le = preprocessing.LabelEncoder()
le.fit(data['category'])
print(list(le.classes_))
print(le.transform(['FOOD & DRINK', 'GROUPS VOICES', 'HOME & LIVING']))

['ARTS & CULTURE', 'BUSINESS & FINANCES', 'COMEDY', 'CRIME', 'DIVORCE', 'EDUCATION', 'ENTERTAINMENT', 'ENVIRONMENT', 'FOOD & DRINK', 'GROUPS VOICES', 'HOME & LIVING', 'IMPACT', 'MEDIA', 'MISCELLANEOUS', 'PARENTING', 'POLITICS', 'RELIGION', 'SCIENCE & TECH', 'SPORTS', 'STYLE & BEAUTY', 'TRAVEL', 'WEDDINGS', 'WEIRD NEWS', 'WELLNESS', 'WOMEN', 'WORLD NEWS']
[ 8  9 10]


In [20]:
data['category encoded'] = le.transform(data['category'])
data.head()

,category,headline,authors,link,short_description,date,category encoded
0,CRIME,"[there, were, 2, mass, shootings, in, texas, l...",Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,"[she, left, her, husband, ., he, killed, their...",2018-05-26,3
1,ENTERTAINMENT,"[will, smith, joins, diplo, and, nicky, jam, f...",Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,"[of, course, it, has, a, song, .]",2018-05-26,6
2,ENTERTAINMENT,"[hugh, grant, marries, for, the, first, time, ...",Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,"[the, actor, and, his, longtime, girlfriend, a...",2018-05-26,6
3,ENTERTAINMENT,"[jim, carrey, blasts, 'castrato, ', adam, schi...",Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,"[the, actor, gives, dems, an, ass-kicking, for...",2018-05-26,6
4,ENTERTAINMENT,"[julianna, margulies, uses, donald, trump, poo...",Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"[the, ``, dietland, '', actress, said, using, ...",2018-05-26,6


### Splitting into train and test



In [21]:
train=data.sample(frac=0.8,random_state=200) #random state is a seed value
test=data.drop(train.index)

train.index= range(len(train))
test.reset_index= range(len(test))


RangeIndex(start=0, stop=160682, step=1)

In [42]:
len(data)

200853

### OUR MODELS

In [36]:
#the code from assignment 2 was changed here. We had a framework to start with this way.

class FeatureCreator:
    def __init__(self):
        self.voc = {}
        self.id = {}
        self.count = 0
    
    def createFeatureSet(self, data):
        # t1 and t2 are the 2 questions we want to compare
        for index, new in data.iterrows():
            
            for w in new['headline']:
                if w in self.voc:
                    self.voc[w] += 1
                else:
                    self.voc[w] = 1
                    self.id[w] = self.count
                    self.count += 1
                    
            for w in new['short_description']:
                if w in self.voc:
                    self.voc[w] += 1
                else:
                    self.voc[w] = 1
                    self.id[w] = self.count
                    self.count += 1
                
        
class BowFeatureCreator(FeatureCreator): #runs in O(n^2)
    
     def createFeatures(self,data):
        id_len=len(self.id) #we will add this length to the index if we want to access the second question. It is made clear in the implementation

        features = dok_matrix((len(data),self.count*2)) #we will have twice the entries. One |VOC| x |2| one for the headline and the other one for the description
        print(features.shape)
        label = []
        i=0
        for i, row in data.iterrows():
            for w in row['headline']:
                if w in self.id:
                    features[i,self.id[w]] += 1
                    #features[i,self.id[w] + self.count] += 1
            for w in row['short_description']:
                if w in self.id:
                    #features[i, self.id[w]]+= 1 #now there is one bow for the headline and another for the description
                    features[i,self.id[w] + self.count] += 1
            label.append(row['category encoded'])
            
            
        #we want to make difference independent of the sentences,
        #that is, no negative values. We must take absolute value of features
        # not the most efficient thing ever here, but it does the job
        """ i=0
        for j, row in data.iterrows():
            for w in row['headline']:
                if w in self.id:
                    if features[j,self.id[w]+self.count]<0:
                        features[j,self.id[w]+self.count] *= -1
            for w in row['short_description']:
                if w in self.id:
                     if features[j,self.id[w]+self.count]<0:
                        features[j,self.id[w] + self.count] *= -1
            i+=1.
            if i%1000 == 0:
                print('iteration: ', i)"""

        #features=self.absoluteValue(features,data)
        return features,label

Calculate size of the vocabulary:

In [37]:
%%time
features=BowFeatureCreator()
features.createFeatureSet(train)

Wall time: 28.9 s


In [38]:
print(len(features.id))
print(len(features.voc))
print(features.count)

#print(train.shape)
print(len(train))

for k,v in features.id.items():
    if v > len(features.id):
        print(k)
        print(v)

105693
105693
105693
160682


In [39]:
f,l = features.createFeatures(train)
print(features.count)

(160682, 211386)
105693


### Multinomial Naive Bayes Classifier

In [40]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(f, l)

MultinomialNB()

### MODEL EVALUATION

We need to create many more variations with many different features

In [44]:
from sklearn.metrics import classification_report
# testf,testl = features.createFeatures(test)
predicted_nb = mnb.predict(f)
print(classification_report(l, predicted_nb));

              precision    recall  f1-score   support

           0       0.97      0.16      0.28      3099
           1       0.81      0.35      0.49      6234
           2       0.93      0.10      0.18      4139
           3       0.85      0.31      0.46      2716
           4       0.99      0.18      0.31      2762
           5       0.90      0.01      0.02      1708
           6       0.59      0.83      0.69     12823
           7       0.92      0.16      0.27      3154
           8       0.90      0.66      0.76      6656
           9       0.72      0.52      0.60      9500
          10       0.97      0.33      0.49      3337
          11       0.96      0.02      0.03      2738
          12       0.96      0.04      0.08      2269
          13       0.96      0.02      0.04      2267
          14       0.60      0.70      0.64     10070
          15       0.55      0.95      0.69     26213
          16       0.98      0.04      0.08      2022
          17       0.96    

# IGNORE FROM HERE ONWARDS

//we will trin the model once we are done with the cleaning

First we split the data such that the proportion of category labels is the same in train and test. Remember, category will be our target

In [ ]:
train, test = train_test_split(data, test_size=0.2, stratify=data['category'])

Leave that for later, do not pay too much attention for now

In [ ]:

class_weights = dict(zip(np.unique(y_train), class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)))

Then you use the class weights during the training process:

train_history = model.fit( train_dataset, steps_per_epoch=n_steps, class_weight=class_weights )

In [ ]:
data.loc[1,'headline']

In [ ]:
sentences=[]
for i, row in data.iterrows():
    sentences.append(row['headline'])

model_embeddings=Word2Vec(vector_size=20, min_count=1)
model_embeddings.build_vocab(sentences, progress_per=10000)
model_embeddings.train(sentences, total_examples=model_embeddings.corpus_count, epochs=30,report_delay=1)
#model_embeddings.build(data['headlines'])


In [ ]:
model_embeddings.wv.most_similar(positive=["korea"])
model_embeddings.wv.most_similar(positive=["Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song"])